# 100k events ; using CKKW

In [55]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0
    initial_events_sq = events_passed_met_cut_sq = events_passed_pt_eta_cut_sq = events_passed_jets_criteria_sq = events_passed_delta_phi_cut_sq = events_passed_leading_jet_cut_sq = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb # https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = {200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    # Initialize dictionary to store squared sum of event weights for each MET cut value
    met_cuts_sq = {met_cut: 0 for met_cut in met_cuts}

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt = leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
        initial_events_sq += (branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights) ** 2

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            events_passed_met_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_pt_eta_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_jets_criteria_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                    events_passed_delta_phi_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                        events_passed_leading_jet_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                                met_cuts_sq[met_cut] += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2
                                

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")

    # Calculate statistical uncertainties for each cut
    initial_events_uncer = np.sqrt(initial_events_sq)
    events_passed_met_cut_uncer = np.sqrt(events_passed_met_cut_sq)
    events_passed_pt_eta_cut_uncer = np.sqrt(events_passed_pt_eta_cut_sq)
    events_passed_jets_criteria_uncer = np.sqrt(events_passed_jets_criteria_sq)
    events_passed_delta_phi_cut_uncer = np.sqrt(events_passed_delta_phi_cut_sq)
    events_passed_leading_jet_cut_uncer = np.sqrt(events_passed_leading_jet_cut_sq)

    print(f"\nInitial number of events: {initial_events:.2f} +/- {initial_events_uncer:.2f} ")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {events_passed_met_cut_uncer:.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {events_passed_pt_eta_cut_uncer:.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {events_passed_jets_criteria_uncer:.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {events_passed_delta_phi_cut_uncer:.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {events_passed_leading_jet_cut_uncer:.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    ratios_uncer = {} # Dictionary to store the ratios uncertainty

    nominal_ratio = None
    max_ratio_met_cut = None
    max_upper_bound = None
    max_lower_bound = None

    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            ratio_uncer = np.sqrt(met_cuts_sq[met_cut]) / paper_met_cuts[met_cut]
            ratios[met_cut] = ratio
            ratios_uncer[met_cut] = ratio_uncer
            lower_bound = ratio - ratio_uncer
            upper_bound = ratio + ratio_uncer
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncer:.2f} with lower bound: {lower_bound:.2f}, and upper bound: {upper_bound:.2f}")
            if nominal_ratio is None or ratio > nominal_ratio:
                nominal_ratio = ratio
                max_ratio_met_cut = met_cut
            if max_lower_bound is None or lower_bound > max_lower_bound:
                max_lower_bound = lower_bound
            if max_upper_bound is None or upper_bound > max_upper_bound:
                max_upper_bound = upper_bound

    # Print the maximum ratio and its MET cut value
    print(f"\nNominal ratio is: {nominal_ratio:.2f} at SR MET > {max_ratio_met_cut} GeV.")
    print("\nThe maximum gluino mass point can be that for which the cross section is around", format(cross_section * (1 / nominal_ratio), '5f'))

    print(f"\nThe maximum lower bound is: {max_lower_bound}")
    print(f"\nThe maximum upper bound is: {max_upper_bound}")

    percentage_difference_lower = ((abs(max_lower_bound - nominal_ratio) / nominal_ratio) * 100)
    print("\npercentage_difference_lower", percentage_difference_lower)
    percentage_difference_upper = ((abs(max_upper_bound - nominal_ratio) / nominal_ratio) * 100)
    print("\npercentage_difference_upper", percentage_difference_upper)

    # Now add cross-section uncertainty

    percentage_difference_lower_xsection_add = math.sqrt(percentage_difference_lower**2 + 11.5**2)
    print("\npercentage_difference_lower_xsection_add", percentage_difference_lower_xsection_add)
    percentage_difference_upper_xsection_add = math.sqrt(percentage_difference_upper**2 + 11.5**2)
    print("\npercentage_difference_upper_xsection_add", percentage_difference_upper_xsection_add)

    print("\nThe cross section for final uncer lower bound is =", format(cross_section * (1 / (nominal_ratio - (percentage_difference_lower_xsection_add/100 * nominal_ratio))), '5f'))
    print("\nThe cross section for final uncer upper bound is =", format(cross_section * (1 / (nominal_ratio + (percentage_difference_upper_xsection_add/100 * nominal_ratio))), '5f'))




# For N1 995 GeV ; gl 1000 GeV

In [56]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-995-1000_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-995-1000_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15051.02 +/- 107.39

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 15024.79 +/- 107.30

Number of events passed number of jets <= 4 cut: 12259.99 +/- 96.65

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7243.71 +/- 74.20

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5848.64 +/- 67.31

For SR MET > 200 GeV: S_Exp: 5848.64, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4864.03, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3770.92, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.18, and upper bound: 0.18

For SR MET > 350 G

# For N1 990 GeV ; gl 1000 GeV

In [57]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15015.02 +/- 107.28

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 14989.51 +/- 107.20

Number of events passed number of jets <= 4 cut: 12277.87 +/- 96.72

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7273.71 +/- 74.36

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5858.45 +/- 67.37

For SR MET > 200 GeV: S_Exp: 5858.45, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4867.22, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3762.93, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.18, and upper bound: 0.18

For SR MET > 350 G

# For N1 950 GeV ; gl 1000 GeV

In [58]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-950-1000_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-950-1000_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15030.31 +/- 107.34

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 15004.98 +/- 107.26

Number of events passed number of jets <= 4 cut: 12293.93 +/- 96.82

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7188.55 +/- 73.92

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5826.43 +/- 67.17

For SR MET > 200 GeV: S_Exp: 5826.43, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4857.66, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3716.45, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.18, and upper bound: 0.18

For SR MET > 350 G

# For N1 590 GeV ; gl 600 GeV

In [59]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-590-600_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.113E+02)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-590-600_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 1582000.00 +/- 5725.23 

Number of events passed MET > 200 GeV cut: 442190.18 +/- 3153.36

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 441534.53 +/- 3151.13

Number of events passed number of jets <= 4 cut: 360821.62 +/- 2840.52

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 215526.41 +/- 2191.07

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 172320.32 +/- 1978.54

For SR MET > 200 GeV: S_Exp: 172320.32, S_Obs: 102274, and ratio: 1.68 +/- 0.02 with lower bound: 1.67, and upper bound: 1.70

For SR MET > 250 GeV: S_Exp: 144014.05, S_Obs: 41158, and ratio: 3.50 +/- 0.04 with lower bound: 3.45, and upper bound: 3.54

For SR MET > 300 GeV: S_Exp: 110507.90, S_Obs: 20893, and ratio: 5.29 +/- 0.08 with lower bound: 5.21, and upper bound: 5

# For N1 965 GeV ; gl 975 GeV

In [60]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-965-975_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.462)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-965-975_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 64680.00 +/- 234.08 

Number of events passed MET > 200 GeV cut: 18133.25 +/- 129.11

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 18100.99 +/- 129.00

Number of events passed number of jets <= 4 cut: 14760.11 +/- 116.15

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 8674.42 +/- 88.90

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 6926.14 +/- 80.22

For SR MET > 200 GeV: S_Exp: 6926.14, S_Obs: 102274, and ratio: 0.07 +/- 0.00 with lower bound: 0.07, and upper bound: 0.07

For SR MET > 250 GeV: S_Exp: 5787.24, S_Obs: 41158, and ratio: 0.14 +/- 0.00 with lower bound: 0.14, and upper bound: 0.14

For SR MET > 300 GeV: S_Exp: 4445.87, S_Obs: 20893, and ratio: 0.21 +/- 0.00 with lower bound: 0.21, and upper bound: 0.22

For SR MET > 350 G

# For N1 850 GeV ; gl 1000 GeV

In [61]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-850-1000_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-850-1000_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 14995.38 +/- 107.19

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 14973.91 +/- 107.11

Number of events passed number of jets <= 4 cut: 12265.48 +/- 96.67

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7205.37 +/- 73.98

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5798.37 +/- 67.00

For SR MET > 200 GeV: S_Exp: 5798.37, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4861.78, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3695.22, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.17, and upper bound: 0.18

For SR MET > 350 G

# For N1 900 GeV ; gl 1000 GeV

In [62]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-900-1000_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-900-1000_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15102.83 +/- 107.56

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 15078.83 +/- 107.48

Number of events passed number of jets <= 4 cut: 12353.09 +/- 97.02

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7208.73 +/- 74.05

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5797.09 +/- 67.05

For SR MET > 200 GeV: S_Exp: 5797.09, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4895.83, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3759.95, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.18, and upper bound: 0.18

For SR MET > 350 G

# For N1 1015 GeV ; gl 1025 GeV

In [63]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-1015-1025_PileUp/Events/run_100k/tag_1_delphes_events.root", 0.322)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-1015-1025_PileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 45080.00 +/- 163.14 

Number of events passed MET > 200 GeV cut: 12576.05 +/- 89.77

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 12554.98 +/- 89.70

Number of events passed number of jets <= 4 cut: 10263.51 +/- 80.89

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 6045.39 +/- 62.02

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 4862.24 +/- 56.16

For SR MET > 200 GeV: S_Exp: 4862.24, S_Obs: 102274, and ratio: 0.05 +/- 0.00 with lower bound: 0.05, and upper bound: 0.05

For SR MET > 250 GeV: S_Exp: 4059.83, S_Obs: 41158, and ratio: 0.10 +/- 0.00 with lower bound: 0.10, and upper bound: 0.10

For SR MET > 300 GeV: S_Exp: 3113.53, S_Obs: 20893, and ratio: 0.15 +/- 0.00 with lower bound: 0.15, and upper bound: 0.15

For SR MET > 350 Ge

# pile-up ATLAS card, but modified values for energy resolution

In [64]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp-ATLAS-default/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp-ATLAS-default/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15032.83 +/- 107.34

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 15020.46 +/- 107.29

Number of events passed number of jets <= 4 cut: 12283.40 +/- 96.76

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7222.73 +/- 74.14

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5801.74 +/- 67.09

For SR MET > 200 GeV: S_Exp: 5801.74, S_Obs: 102274, and ratio: 0.06 +/- 0.00 with lower bound: 0.06, and upper bound: 0.06

For SR MET > 250 GeV: S_Exp: 4845.02, S_Obs: 41158, and ratio: 0.12 +/- 0.00 with lower bound: 0.12, and upper bound: 0.12

For SR MET > 300 GeV: S_Exp: 3725.27, S_Obs: 20893, and ratio: 0.18 +/- 0.00 with lower bound: 0.18, and upper bound: 0.18

F